In [14]:
from mindquantum import *
import numpy as np
import scipy.sparse as sp
from scipy.optimize import minimize 
import matplotlib.pyplot as plt 

In [15]:
class VariationalGrover(object):
    def __init__(self, n_qubits, alpha, order=0):
        self.n = n_qubits
        self.a = alpha 
        self.order = order

    def oracle(self, phase):                         # V_C                  
        list_ = range(self.n-self.a, self.n-1)
        circ = PhaseShift(phase).on(self.n-1,list(list_))
        return circ 
    
    def poperator(self, phase):                      # V_B
        circ = UN(H,self.n)
        circ += UN(X,self.n)
        circ += PhaseShift(phase).on(self.n-1,list(range(self.n-1)))
        circ += UN(X,self.n)
        circ += UN(H,self.n)
        return circ
    
    def order_mixer(self, phase):                   # order V_B
        circ1 = Circuit()
        for i in range(self.n):
            circ1 += RX(phase).on(i)
            circ1 += RX(phase).on(i)
        if self.order == 1:
            return circ1 
        elif self.order == 2:
            circ2 = Circuit()
            for i in range(self.n-1):
                for j in range(i, self.n):
                    circ2 += Rxx(phase).on([i,j])
            return circ1+circ2 

    def hamiltonian(self):
        list_left = [1]*((2**self.a-1)*2**(self.n-self.a))
        list_right = [0]*(2**(self.n-self.a))
        data = list_left+list_right
        row = list(range(2**self.n))
        col = list(range(2**self.n))
        csr = sp.csr_matrix((data, (row,col)), shape=(2**self.n,2**self.n))
        return Hamiltonian(csr)
    
    def cost_function(self,params):
        circ = UN(H,self.n)
        for i in range(int(len(params)/2)):
            circ += self.oracle(f"phi{i}")
            if self.order == 0:
                circ += self.poperator(f"theta{i}")
            else:
                circ += self.order_mixer(f"theta{i}")
        sim = Simulator("projectq", self.n)
        ops = sim.get_expectation_with_grad(self.hamiltonian(), circ)
        f, g = ops(params)
        return np.squeeze(f.real), np.squeeze(g.real)
    

In [16]:
cost0 = []
cost1 = []
cost2 = []
params0 = []
params1 = []
params2 = []
for p in range(1,13):
    fun=1
    for i in range(30):
        params_temp = np.random.rand(2*p)
        res = minimize(VariationalGrover(6,6,order=0).cost_function, params_temp, method="SLSQP")
        if res.fun <= fun:
            fun = res.fun
            params = res.x 
            print("fun", fun, "temp", i, "order", 0)
    cost0.append(fun)
    params0.append(params)

for p in range(1,13):
    fun=1
    for i in range(30):
        params_temp = np.random.rand(2*p)
        res = minimize(VariationalGrover(6,6,order=1).cost_function, params_temp, method="SLSQP")
        if res.fun <= fun:
            fun = res.fun
            params = res.x 
            print("fun", fun, "temp", i, "order", 1)
    cost1.append(fun)
    params1.append(params)

for p in range(1,13):
    fun=1
    for i in range(30):
        params_temp = np.random.rand(2*p)
        res = minimize(VariationalGrover(6,6,order=2).cost_function, params_temp, method="SLSQP")
        if res.fun <= fun:
            fun = res.fun
            params = res.x 
            print("fun", fun, "temp", i, "order", 2)
    cost2.append(fun)
    params2.append(params)




    


fun 0.8651733472158909 temp 0 order 0
fun 0.8651733437524194 temp 5 order 0
fun 0.8651733398439814 temp 7 order 0
fun 0.6561049648317767 temp 0 order 0
fun 0.65610490058934 temp 3 order 0
fun 0.6561048210128495 temp 4 order 0
fun 0.6561048098791711 temp 12 order 0
fun 0.40862515011237444 temp 0 order 0
fun 0.4086199193481944 temp 1 order 0
fun 0.40861988486274997 temp 16 order 0
fun 0.1836241459547762 temp 0 order 0
fun 0.1836236060466254 temp 1 order 0
fun 0.183623378367056 temp 7 order 0
fun 0.18362314048716177 temp 8 order 0
fun 0.18362304559073042 temp 10 order 0
fun 0.0364849419872447 temp 0 order 0
fun 0.03648463450534235 temp 9 order 0
fun 2.798198625392614e-07 temp 0 order 0
fun 1.2656215812018903e-07 temp 1 order 0
fun 1.1850283954819917e-07 temp 4 order 0
fun 1.0064179435665695e-07 temp 5 order 0
fun 6.797120965210483e-08 temp 7 order 0
fun 1.920781422725474e-08 temp 8 order 0
fun 1.3068085676477183e-08 temp 9 order 0
fun 7.578459922405493e-09 temp 18 order 0
fun 7.3523998353

In [ ]:
x=range(1,13)
plt.plot(x, cost1,linestyle=":", linewidth=1.5, label="order = 1",marker="o", markerfacecolor="none")
plt.plot(x, cost2,linestyle=":", linewidth=1.5, label="order = 2",marker="o",markerfacecolor="none")
plt.plot(x, cost0,linestyle="-", linewidth=1.5, label="oder = 0",marker="o", color='indianred',markerfacecolor="none")
plt.axis([1,12,0,1])
plt.xlabel(r'depth $p$',fontsize=13)
plt.ylabel(r'$C_p$',fontsize=13)
plt.legend()
#plt.plot(x, y_p_3,linestyle="-",marker="o")
plt.savefig("order_mixer.png")

plt.show()

In [ ]:
cost0

[array(0.86517336),
 array(0.86517336),
 array(0.86517335),
 array(0.86517335),
 array(0.86517335),
 array(0.86517335),
 array(0.86517335),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
 array(0.86517334),
